### Vector stores and retrievers

This video tutorial will familiarize you with LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

We will cover

- Documents
- Vector stores
- Retrievers


### Documents

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
  The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:


In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10b7eddd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10ba00390>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/shankii/Documents/AI/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
## VectorStores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embedding=embeddings)
vectorstore

In [13]:
vectorstore.similarity_search("cat")

[Document(id='54e2184a-d6c1-4f9d-9d5f-55b868224f19', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='2043c7f3-4444-4606-916c-b712ce45c5f8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='de52cafb-17de-4ec3-bbbd-5654d7f92d30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='6a8a4352-0dab-4e44-8c86-a9b5974a6b7f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [14]:
## Async query
# This demonstrates the asynchronous version of similarity search
# It performs the same search as similarity_search() but can be awaited
# Useful when you want non-blocking behavior in async applications

await vectorstore.asimilarity_search("cat")

[Document(id='54e2184a-d6c1-4f9d-9d5f-55b868224f19', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='2043c7f3-4444-4606-916c-b712ce45c5f8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='de52cafb-17de-4ec3-bbbd-5654d7f92d30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='6a8a4352-0dab-4e44-8c86-a9b5974a6b7f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [15]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='54e2184a-d6c1-4f9d-9d5f-55b868224f19', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351053833961487),
 (Document(id='2043c7f3-4444-4606-916c-b712ce45c5f8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351053833961487),
 (Document(id='de52cafb-17de-4ec3-bbbd-5654d7f92d30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852),
 (Document(id='6a8a4352-0dab-4e44-8c86-a9b5974a6b7f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852)]

### Retrievers

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:


In [ ]:
# Import required types and classes
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# Create a retriever by wrapping the vectorstore's similarity_search method in a RunnableLambda
# - RunnableLambda converts the similarity_search function into a runnable object that can be used in a LangChain pipeline.
# - bind(k=1) sets the default number of results to return to 1
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

# Test the retriever by doing a batch search for "cat" and "dog"
## batch: Allows you to run the retriever on multiple queries at once.
retriever.batch(["cat", "dog"])

[[Document(id='2043c7f3-4444-4606-916c-b712ce45c5f8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='de52cafb-17de-4ec3-bbbd-5654d7f92d30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:


In [17]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})
retriever.batch(["cat", "dog"])

[[Document(id='2043c7f3-4444-4606-916c-b712ce45c5f8', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='de52cafb-17de-4ec3-bbbd-5654d7f92d30', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [19]:
## RAG (Retrieval Augmented Generation)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# In RAG, we need context from our knowledge base (vectorstore) to:
# 1. Ground the LLM's responses in factual information
# 2. Provide up-to-date or domain-specific knowledge
# 3. Reduce hallucination by having the LLM only use provided context

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability using only the provided context.",
        ),
        (
            "human",
            """
                Context: {context}

                Question: {question} 
            """,
        ),
    ]
)

# Chain that:
# 1. Takes a question
# 2. Retrieves relevant context from vectorstore
# 3. Combines context + question in prompt
# 4. Sends to LLM for response
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about dogs")
print(response.content)

According to the document, dogs are great companions, known for their loyalty and friendliness.
